In [1]:
import cards
from typing import List

cards.LOGGING_ENABLED = True

def print_tree(state:cards.Player, depth = 0):
    print ("  "*depth, state.short_str())
    for child in state.childstates:
        print_tree(child, depth+1)


def get_all_leaf_nodes(state:cards.Player) -> List[cards.Player]:
    if state.is_pruned:
        return []

    if len(state.childstates) == 0:
        return [state]
    else:
        leaf_nodes = []
        for child in state.childstates:
            leaf_nodes.extend(get_all_leaf_nodes(child))
        return leaf_nodes


In [2]:
decklist = """
1 Arboreal Grazer
1 Llanowar Elves
1 Elvish Mystic
1 Sol Ring
1 Arbor Elf
2 Sakura-Tribe Elder
4 Wall of Roots
4 Chancellor of the Tangle
1 Panglacial Wurm
1 Abundant Harvest
4 Ancient Stirrings
4 Caravan Vigil
2 Lay of the Land
4 Reclaim the Wastes
3 Explore
4 Land Grant
3 Rampant Growth
2 Nissa's Pilgrimage
4 Recross the Paths
2 Search for Tomorrow
4 Goblin Charbelcher
4 Wild Growth
7 Forest
"""

In [3]:

# Create a player with a predictable seed
player = cards.Player(decklist, 12)
player.start_game()

player.start_turn()
# Ensure that the player has 7 cards in hand
assert len(player.hand) == 7, f"Player should have 7 cards in hand, has {len(player.hand)}"

# Get a Search for Tomorrow
search = player.debug_force_get_card_in_hand("Search for Tomorrow")

# Cast the card for its alternate cost
player.mana_pool += search.alt_cost # Cheat and add mana to pay costs
player.alt_play(search)

# Pass the turn
player.start_turn()
# Ensure that we still have no lands
assert player.lands == 0
assert player.mana_pool == 0

# Pass the turn again
player.start_turn()
# Ensure that we now have 1 land, and it is untapped
assert player.lands == 1
assert player.mana_pool == 1

# Now test casting Search for Tomorrow for its full cost.
search = player.debug_force_get_card_in_hand("Search for Tomorrow")
print(f'player.has_mana: {player.has_mana(search.cost, search.colorless_cost)}')
player.mana_pool += search.cost
player.play(search)
# Ensure that we now have 2 lands, both untapped
assert player.lands == 2
assert player.mana_pool == 2


player.has_mana: False


In [4]:

# Create a player with a predictable seed
player = cards.Player(decklist, 12)
player.start_game()
player.start_turn()
# Ensure that the player has 7 cards in hand
assert len(player.hand) == 7

# Get a Sol Ring
solring = player.debug_force_get_card_in_hand("Sol Ring")
belcher = player.debug_force_get_card_in_hand("Goblin Charbelcher")
search = player.debug_force_get_card_in_hand("Search for Tomorrow")

# Cast the Sol Ring
player.mana_pool += 2 # Cheat and add mana to pay costs
player.play(solring)

assert player.colorless_lands == 2
assert player.colorless_mana_pool == 2

# Cast the Search for Tomorrow
player.play(search)

# Ensure that we now have 1 land, and it is untapped.
assert player.mana_pool == 1
assert player.colorless_mana_pool == 0

# Pass the turn
player.start_turn()

assert player.lands == 1
assert player.mana_pool == 1
assert player.colorless_mana_pool == 2

# Cheat and add some mana
player.mana_pool += 4
player.play(belcher)

# Ensure that we spent our colorless mana before our colored mana
assert player.mana_pool == 3
assert player.colorless_mana_pool == 0


In [5]:

# Create a player with a predictable seed
player = cards.Player(decklist, 14)
player.start_game()
player.start_turn()
print(len(player.hand))
# Ensure that the player has 7 cards in hand
assert len(player.hand) == 7

# Get a Land Grant
card = player.debug_force_get_card_in_hand("Land Grant")

print(card.long_str(player))

# Cast the card for its alternate cost
assert player.hand.count_cards("Forest") == 0

assert not card.can_play(player)
assert card.can_alt_play(player)

player.alt_play(card)

assert player.hand.count_cards("Forest") == 1


7
Land Grant [2]   Can play: False / True   Can activate: False


In [6]:
# Test the AI's ability to cast Land Grant when needed

# Create a player with a predictable seed
player = cards.Player(decklist, 14)
player.start_game()
player.start_turn()
# Ensure that the player has 7 cards in hand
assert len(player.hand) == 7

# Get a Search for Tomorrow
card = player.debug_force_get_card_in_hand("Land Grant")

print(f'Starting state:')
print(player)
print()

player.step_next_actions()

for i in range(5):
    leafs = get_all_leaf_nodes(player)
    for leaf in leafs:
        if leaf.current_turn == 1:
            leaf.step_next_actions()

print_tree(player)

leafs = get_all_leaf_nodes(player)
for i, leaf in enumerate(leafs):
    print()
    print('********')
    print(f'Leaf #{i}: {leaf.short_str()}')
    print()
    leaf.dumplog()
    print()
    print(leaf)

Starting state:
Turn [1] - Opponent Life: 20
 7 Forests in library
 Lands: 1 / 0  (1 drops avail.)
 Colorless: 0 / 0
 Hand: 8 cards
  Chancellor of the Tangle
  Explore
  Land Grant
  Nissa's Pilgrimage
  Panglacial Wurm
  Rampant Growth
  Recross the Paths
  Search for Tomorrow
 Table: 0 cards
 Graveyard: 0 cards
 Library: 56 cards
 Exile: 0 cards


 1)  LID: 7  H: 8  Mana: 1/0 (0/0) [0]  OLife: 20 '  Chancellor of the Tangle adding 1 to mana pool'
   1)  LID: 6  H: 8  Mana: 1/0 (0/0) [1]  OLife: 20 ' Alt play: Land Grant'
     1)  LID: 6  H: 7  Mana: 2/1 (0/0) [0]  OLife: 20 ' Play: Forest'
       1)  LID: 5  H: 6  Mana: 0/2 (0/0) [0]  OLife: 20 ' Play: Rampant Growth'
         2)  LID: 5  H: 7  Mana: 2/2 (0/0) [0]  OLife: 20 ' Draw 1 card(s)'
       1)  LID: 6  H: 6  Mana: 1/1 (0/0) [0]  OLife: 20 ' Alt play: Search for Tomorrow'
         2)  LID: 5  H: 7  Mana: 1/1 (0/0) [1]  OLife: 20 ' Draw 1 card(s)'
       1)  LID: 5  H: 7  Mana: 0/1 (0/0) [1]  OLife: 20 ' Draw 1 card(s)'
     

In [7]:
player.dumplog()
print()
print(player)


 Draw 7 card(s)
Beginning turn 1: 1)  LID: 7  H: 7  Mana: 0/0 (0/0) [0]  OLife: 20 ' Draw 7 card(s)'
  Chancellor of the Tangle adding 1 to mana pool

Turn [1] - Opponent Life: 20
 7 Forests in library
 Lands: 1 / 0  (1 drops avail.)
 Colorless: 0 / 0
 Hand: 8 cards
  Chancellor of the Tangle
  Explore
  Land Grant
  Nissa's Pilgrimage
  Panglacial Wurm
  Rampant Growth
  Recross the Paths
  Search for Tomorrow
 Table: 0 cards
 Graveyard: 0 cards
 Library: 56 cards
 Exile: 0 cards



In [10]:
cards_by_type = {
    "Creature": [],
    "Artifact": [],
    "Enchantment": [],
    "Land": [],
    "Spell": []
}

card_subclasses = cards.Card.__subclasses__()

# Sort all by mana cost, then by name
card_subclasses = sorted(card_subclasses, key=lambda x: (x.cost, x.name))

# Now when we iterate through, they should already be sorted by mana cost and name
for subclass in card_subclasses:
    if subclass.cardtype in cards_by_type:
        cards_by_type[subclass.cardtype].append(subclass)
    else:
        cards_by_type["Spell"].append(subclass)

# Print out each list
for cardtype in cards_by_type:
    print(f'')
    print(f'### {cardtype}s:')
    last_cmc = 0
    for card in cards_by_type[cardtype]:
        # Skip cards that we aren't playing right now
        if card.deck_max_quant == 0:
            continue

        if card.cost != last_cmc:
            print(f'') # # CMC: {card.cost}')
            last_cmc = card.cost
        print(f"0 {card.name}")








### Creatures:

0 Arbor Elf
0 Arboreal Grazer
0 Elvish Mystic
0 Krosan Wayfarer
0 Llanowar Elves

0 Sakura-Tribe Elder
0 Tangled Florahedron
0 Wall of Roots

0 Elvish Spirit Guide
0 Simian Spirit Guide

0 Generous Ent

0 Beanstalk Giant
0 Chancellor of the Tangle
0 Panglacial Wurm

### Artifacts:

0 Sol Ring

0 Goblin Charbelcher

### Enchantments:

0 Wild Growth

### Lands:
0 Forest

### Spells:

0 Abundant Harvest
0 Ancient Stirrings
0 Caravan Vigil
0 Lay of the Land
0 Reclaim the Wastes

0 Edge of Autumn
0 Explore
0 Land Grant
0 Manamorphose
0 Nissa's Triumph
0 Rampant Growth

0 Beneath the Sands
0 Cultivate
0 Grow from the Ashes
0 Nissa's Pilgrimage
0 Recross the Paths
0 Search for Tomorrow

0 Migration Path
